In [1]:
from openai import OpenAI

In [4]:
from dotenv import load_dotenv

In [5]:
load_dotenv()

True

In [7]:
import os

In [8]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [13]:
response = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        {
            "role": "system", "content": f"""
            You are an expert in categorising IT support tickets. Given the support ticket, you need categorise each ticket into one of the 
            categories listed below:
            1. Hardware
            2. Software
            3. Others
    
            Response in one of the above words only. 
            """},
        {
            "role": "user", "content": f"My monitor won't turn on"
        }
    ], temperature = 0
)

print(response.choices[0].message.content)

Hardware


In [16]:
eval_test = client.evals.create(
    name = "IT Ticket Categorisation",
    data_source_config = {
        "type": "custom",
        "item_schema": {
            "type": "object",
            "properties": {
                "ticket_text": {"type": "string"},
                "correct_label": {"type": "string"}
            },
            "required": ["ticket_text", "correct_label"]
        },
        "include_sample_schema": True
    },

    testing_criteria = [
        {
            "type": "string_check",
            "name": "Match to human label",
            "input": "{{sample.output_text}}",
            "operation": "eq",
            "reference": "{{item.correct_label}}"
        }
    ]
)

print(eval_test)

EvalCreateResponse(id='eval_6940b036ee948191a812f11f980d4935', created_at=1765847094, data_source_config=EvalCustomDataSourceConfig(schema_={'type': 'object', 'properties': {'item': {'type': 'object', 'properties': {'ticket_text': {'type': 'string'}, 'correct_label': {'type': 'string'}}, 'required': ['ticket_text', 'correct_label']}, 'sample': {'type': 'object', 'properties': {'model': {'type': 'string'}, 'choices': {'type': 'array', 'items': {'type': 'object', 'properties': {'message': {'type': 'object', 'properties': {'role': {'type': 'string', 'enum': ['assistant']}, 'content': {'type': ['string', 'array', 'null']}, 'refusal': {'type': ['boolean', 'null']}, 'tool_calls': {'type': ['array', 'null'], 'items': {'type': 'object', 'properties': {'type': {'type': 'string', 'enum': ['function']}, 'function': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'arguments': {'type': 'string'}}, 'required': ['name', 'arguments']}, 'id': {'type': 'string'}}, 'required': ['type', 'fun

In [45]:
file_upload = client.files.create(
    file = open("/Users/Rakshit.Lodha/evals/tickets.jsonl","rb"),
    purpose = "evals"
)

print(file_upload)

FileObject(id='file-Uimp2cARGPuGZDoEYWdF7G', bytes=268, created_at=1766018864, filename='tickets.jsonl', object='file', purpose='evals', status='processed', expires_at=None, status_details=None)


In [17]:
eval_run = client.evals.runs.create(
    "eval_6940b036ee948191a812f11f980d4935",
    name = "Categorisation",
    data_source = {
        "type": "responses",
        "model": "gpt-3.5-turbo",
        "input_messages": {
            "type": "template",
            "template": [
                {"role": "developer", "content": "You are an expert in categorizing IT support tickets. Given the support ticket below, categorize the request into one of 'Hardware', 'Software', or 'Other'. Respond with only one of those words."},
                {"role": "user", "content": "{{ item.ticket_text }}"},
            ]
        },
        "source": {"type": "file_id", "id": "file-BW89c77fZFoMrq8dG4Xb3k"}
    }
)

print(eval_run)

RunCreateResponse(id='evalrun_6940b117c0f481919adf813d0d40f93f', created_at=1765847319, data_source=DataSourceResponses(source=DataSourceResponsesSourceFileID(id='file-BW89c77fZFoMrq8dG4Xb3k', type='file_id'), type='responses', input_messages=DataSourceResponsesInputMessagesTemplate(template=[DataSourceResponsesInputMessagesTemplateTemplateEvalItem(content="You are an expert in categorizing IT support tickets. Given the support ticket below, categorize the request into one of 'Hardware', 'Software', or 'Other'. Respond with only one of those words.", role='developer', type='message'), DataSourceResponsesInputMessagesTemplateTemplateEvalItem(content='{{ item.ticket_text }}', role='user', type='message')], type='template'), model='gpt-3.5-turbo', sampling_params=None), error=None, eval_id='eval_6940b036ee948191a812f11f980d4935', metadata={}, model='gpt-3.5-turbo', name='Categorisation', object='eval.run', per_model_usage=None, per_testing_criteria_results=None, report_url='https://platfo

In [32]:
response_1 = client.chat.completions.create(
    model = "gpt-4o",
    messages = [
        {"role": "system", "content": f""" 
        You are a helpful advisor, who is supposed to answer life's questions thoughtfully and empathetically.
        But at the same time ensure that you're not being too generic and giving actionable advice to the user

        Make sure the answer is conscise and does not hold back 
        """},
        {"role": "user", "content": "Is it wrong to delay marriage because I want to focus on my career right now?"}
    ], temperature = 0.7
)

print(response_1.choices[0].message.content)

It's not wrong at all to delay marriage to focus on your career. Prioritizing your personal and professional goals is a valid choice and can lead to greater fulfillment in the long run. It's vital to communicate openly with your partner about your intentions and ensure you're both aligned in your life goals. Consider setting a realistic timeline for when you might revisit the idea of marriage, and be mindful of balancing your career ambitions with nurturing your relationship to ensure both areas receive the attention they deserve.


In [47]:
eval_test_1 = client.evals.create(
    name = "Helpful assistant",
    data_source_config = {
        "type": "custom",
        "item_schema": {
            "type": "object",
            "properties": {
                "question": {"type": "string"},
                "ideal_answer": {"type": "string"}
            },
            "required": ["question","ideal_answer"]
        },
        "include_sample_schema": True
    },
    testing_criteria = [
        {
            "type": "score_model",
            "name": "Model Score",
            "input": [
                {
                    "role": "system",
                    "content": """You are an impartial and strict evaluation judge.

                    Evaluate whether {{sample.output_text}} satisfies the behavioral intent
                    described in {{item.ideal_answer}}.
                    
                    You must apply the following rubric EXACTLY.
                    
                    CRITERIA DEFINITIONS (DO NOT INFER GENEROUSLY):
                    
                    Empathy (Required for any score > 0)
                    - Explicitly names or reflects the user's emotional state or conflict.
                    - Polite tone alone is insufficient.
                    
                    Concise
                    - <= 150 words.
                    - No more than 3 actionable suggestions.
                    - No motivational filler or repetition.
                    
                    Actionable
                    - At least one concrete, realistic next step.
                    - Abstract advice without execution guidance does NOT qualify.
                    
                    SCORING LOGIC:
                    
                    - If Empathy is missing → score = 0 immediately.
                    - If Empathy is present:
                        - 3 criteria satisfied → score = 5
                        - Exactly 2 satisfied → score = 3
                        - Fewer than 2 → score = 0
                    
                    AUTOMATIC FAILURES (score = 0 regardless of above):
                    - Absolute or prescriptive advice
                    - Moral judgment
                    - Ignoring stated constraints
                    - Forced optimism or dismissive language
                    
                    IMPORTANT:
                    - Do NOT reward generic competence.
                    - Do NOT assume intent.
                    - When uncertain, score lower.
                    - If the response uses generic phrasing that could apply to many unrelated situations
                    (e.g., identical structure, repeated advice templates), score = 3 max.

                    
                    Output ONLY one integer from {0, 3, 5}.

                    """
                },
                {
                    "role": "user",
                    "content": "Reference: {{item.ideal_answer}}. Model Output: {{sample.output_text}}"
                }
            ],
            "pass_threshold": 3,
            "model": "gpt-4o",
            "range": [0,5]
        }
    ]
)

print(eval_test_1)

EvalCreateResponse(id='eval_6943502418d08191bfc81b31087733f9', created_at=1766019108, data_source_config=EvalCustomDataSourceConfig(schema_={'type': 'object', 'properties': {'item': {'type': 'object', 'properties': {'question': {'type': 'string'}, 'ideal_answer': {'type': 'string'}}, 'required': ['question', 'ideal_answer']}, 'sample': {'type': 'object', 'properties': {'model': {'type': 'string'}, 'choices': {'type': 'array', 'items': {'type': 'object', 'properties': {'message': {'type': 'object', 'properties': {'role': {'type': 'string', 'enum': ['assistant']}, 'content': {'type': ['string', 'array', 'null']}, 'refusal': {'type': ['boolean', 'null']}, 'tool_calls': {'type': ['array', 'null'], 'items': {'type': 'object', 'properties': {'type': {'type': 'string', 'enum': ['function']}, 'function': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'arguments': {'type': 'string'}}, 'required': ['name', 'arguments']}, 'id': {'type': 'string'}}, 'required': ['type', 'function', 

In [51]:
file_upload_1 = client.files.create(
    file = open("/Users/Rakshit.Lodha/evals/response.jsonl","rb"),
    purpose = "evals"
)

print(file_upload_1)

FileObject(id='file-RDTn3NCY6piYyznQyCYkiZ', bytes=4382, created_at=1766019441, filename='response.jsonl', object='file', purpose='evals', status='processed', expires_at=None, status_details=None)


In [43]:
eval_run_1 = client.evals.runs.create(
    "eval_6943486408e08191b8cc4b64a88d9867",
    name = "Model Response Check V1",
    data_source = {
        "type": "responses",
        "model": "gpt-4o",
        "input_messages": {
            "type": "template",
            "template": [
                {"role": "developer", "content": """ You are a helpful advisor, 
                who is supposed to answer life's questions thoughtfully and empathetically.
                But at the same time ensure that you're not being too generic and giving actionable advice to the user
                Make sure the answer is conscise and does not hold back """},
                {"role": "user", "content": "{{item.question}}"}
            ]
        },
        "source": {"type": "file_id", "id": "file-QPbtrPmabv46T6BEv9RAXC"}
    }
)


print(eval_run_1)

RunCreateResponse(id='evalrun_6943487312b8819192c82f3f856e11e7', created_at=1766017139, data_source=DataSourceResponses(source=DataSourceResponsesSourceFileID(id='file-QPbtrPmabv46T6BEv9RAXC', type='file_id'), type='responses', input_messages=DataSourceResponsesInputMessagesTemplate(template=[DataSourceResponsesInputMessagesTemplateTemplateEvalItem(content=" You are a helpful advisor, \n                who is supposed to answer life's questions thoughtfully and empathetically.\n                But at the same time ensure that you're not being too generic and giving actionable advice to the user\n                Make sure the answer is conscise and does not hold back ", role='developer', type='message'), DataSourceResponsesInputMessagesTemplateTemplateEvalItem(content='{{item.question}}', role='user', type='message')], type='template'), model='gpt-4o', sampling_params=None), error=None, eval_id='eval_6943486408e08191b8cc4b64a88d9867', metadata={}, model='gpt-4o', name='Model Response Ch

In [39]:
#Failure:

#Is it wrong to delay marriage because I want to focus on my career right now?

#Delaying marriage to focus on your career is not wrong. 
#It's important to prioritize what feels right for you at this stage. 
#Communicate clearly with your partner about your goals and timeline. 
#Make plans to periodically reassess your priorities together, ensuring both your personal and relationship needs are addressed.


#1. First, assess if empathy is present. 
#The model output acknowledges the user's subjective feeling by saying it's important to prioritize what feels right for them. 
#This addresses an emotional aspect of the issue.
#Empathy is present.
#2. Next, check for conciseness. 
#The response is 53 words long, offers no more than 3 actionable suggestions, and does not include motivational filler or repetition.
#Conciseness is satisfied.
#3. Examine for actionable steps. 
#The model suggests "communicate clearly with your partner about your goals and timeline," which is a concrete and realistic next step.
#Actionable is satisfied.
#4. Verify if there are any automatic failures such as moral judgment or prescriptive advice. 
#The response states that delaying marriage is not wrong, which is a prescriptive statement, 
#and therefore represents a moral judgment by deeming the decision categorically acceptable. This violates the criteria.
#The response contains moral judgment, an automatic failure.


#Prescriptive Advice

In [38]:
#Failure:

# Question:

# I have two job offers: one pays more but is very stressful, the other pays less but offers better learning. What should I do?


# Answer:

# Choose based on your current priorities and long-term goals. 
# If financial stability or savings for a specific goal is your top priority, 
# and you're equipped to handle the stress, the higher-paying job might be worth it. 
# However, if you're looking to grow, gain valuable skills, and ultimately increase your career potential, 
# the learning-focused job could offer greater long-term benefits. 
# Consider your personal stress tolerance and how each job aligns with your life aspirations, then decide.


# 1. First, I need to check if empathy is present in the response. 
# The response mentions recognizing the trade-off between financial stability and learning in terms of career. 
# However, it doesn't reflect the user's emotional state or conflict directly. 
# Polite tone or sensible advice is not considered sufficient for empathy according to the rubric.
# Empathy is missing.
# 2. Since empathy was not present, I should assign a score of 0 immediately regardless of other criteria satisfaction.
# The score is 0.


#Lack of empathy

In [40]:
# Is it wrong to delay marriage because I want to focus on my career right now?

# Not at all. Prioritizing your career can be a smart decision if that's where your focus and passion currently lie. 
# Communication with your partner is key. 
# Discuss your goals, and ensure both of you are aligned or able to compromise on timelines. 
# Balance is crucial. Make time to nurture your relationship even as you pursue career goals.

# First, I need to determine if the response shows empathy by explicitly naming or reflecting a user's emotional state or conflict. 
# The response acknowledges the potential conflict between career focus and nurturing a relationship 
# but does not explicitly name or delve into any emotional conflict or state the user may face in this situation.
# Empathy is missing as the response does not explicitly reflect an emotional state or conflict.

#Lack of empathy

In [44]:
eval_run_2 = client.evals.runs.create(
    "eval_6943486408e08191b8cc4b64a88d9867",
    name = "Model Response Check V2",
    data_source = {
        "type": "responses",
        "model": "gpt-4o",
        "input_messages": {
            "type": "template",
            "template": [
                {"role": "developer", "content": """ You are a thoughtful life advisor. 

                Rules that you MUST follow:
                
                1. Always begin by explicitly acknowledging the user's emotional state or inner conflict. 
                Name the tension or uncertainty that they are experiencing. Do not start with advice.
                2. Do not frame the situation as right or wrong, smart or foolish, or correct or incorrect. 
                Avoid absolute judgments and conclusions.
                3. Avoid phrases like: "Not Wrong!", "The Right Choice!", or "A Smart Decision". Frame decisions as trade-offs. 
                Present at least two valid perspectives. Keep the final choice with the user.
                4. Provide 1-3 concrete, realistic next steps. Avoid vague advice like: "Reflect" unless paired with an action. Do not overwhelm. 
                5. Be concise (<150-200 words) with no motivational filler or forced positivity

                The goal is to help the user to think clearly, not to decide for them. 
                """},
                {"role": "user", "content": "{{item.question}}"}
            ]
        },
        "source": {"type": "file_id", "id": "file-QPbtrPmabv46T6BEv9RAXC"}
    }
)


print(eval_run_2)

RunCreateResponse(id='evalrun_6943487a0d888191a2028c0b6860ef23', created_at=1766017146, data_source=DataSourceResponses(source=DataSourceResponsesSourceFileID(id='file-QPbtrPmabv46T6BEv9RAXC', type='file_id'), type='responses', input_messages=DataSourceResponsesInputMessagesTemplate(template=[DataSourceResponsesInputMessagesTemplateTemplateEvalItem(content=' You are a thoughtful life advisor. \n\n                Rules that you MUST follow:\n                \n                1. Always begin by explicitly acknowledging the user\'s emotional state or inner conflict. \n                Name the tension or uncertainty that they are experiencing. Do not start with advice.\n                2. Do not frame the situation as right or wrong, smart or foolish, or correct or incorrect. \n                Avoid absolute judgments and conclusions.\n                3. Avoid phrases like: "Not Wrong!", "The Right Choice!", or "A Smart Decision". Frame decisions as trade-offs. \n                Present at

In [52]:
eval_run_3 = client.evals.runs.create(
    "eval_6943486408e08191b8cc4b64a88d9867",
    name = "Model Response Check V3 with more responses",
    data_source = {
        "type": "responses",
        "model": "gpt-4o",
        "input_messages": {
            "type": "template",
            "template": [
                {"role": "developer", "content": """ You are a thoughtful life advisor. 

                Rules that you MUST follow:
                
                1. Always begin by explicitly acknowledging the user's emotional state or inner conflict. 
                Name the tension or uncertainty that they are experiencing. Do not start with advice.
                2. Do not frame the situation as right or wrong, smart or foolish, or correct or incorrect. 
                Avoid absolute judgments and conclusions.
                3. Avoid phrases like: "Not Wrong!", "The Right Choice!", or "A Smart Decision". Frame decisions as trade-offs. 
                Present at least two valid perspectives. Keep the final choice with the user.
                4. Provide 1-3 concrete, realistic next steps. Avoid vague advice like: "Reflect" unless paired with an action. Do not overwhelm. 
                5. Be concise (<150-200 words) with no motivational filler or forced positivity

                The goal is to help the user to think clearly, not to decide for them. 
                """},
                {"role": "user", "content": "{{item.question}}"}
            ]
        },
        "source": {"type": "file_id", "id": "file-RDTn3NCY6piYyznQyCYkiZ"}
    }
)


print(eval_run_2)

RunCreateResponse(id='evalrun_6943487a0d888191a2028c0b6860ef23', created_at=1766017146, data_source=DataSourceResponses(source=DataSourceResponsesSourceFileID(id='file-QPbtrPmabv46T6BEv9RAXC', type='file_id'), type='responses', input_messages=DataSourceResponsesInputMessagesTemplate(template=[DataSourceResponsesInputMessagesTemplateTemplateEvalItem(content=' You are a thoughtful life advisor. \n\n                Rules that you MUST follow:\n                \n                1. Always begin by explicitly acknowledging the user\'s emotional state or inner conflict. \n                Name the tension or uncertainty that they are experiencing. Do not start with advice.\n                2. Do not frame the situation as right or wrong, smart or foolish, or correct or incorrect. \n                Avoid absolute judgments and conclusions.\n                3. Avoid phrases like: "Not Wrong!", "The Right Choice!", or "A Smart Decision". Frame decisions as trade-offs. \n                Present at